In [ ]:
import os
import logging
from typing import Generator
import httpx
from deepagents import create_deep_agent
from langchain_openai import ChatOpenAI
from openai import APIStatusError

proxy_url = os.getenv("PROXY_URL") or ""
timeout = httpx.Timeout(120.0, connect=10.0, read=120.0, write=120.0)
http_client = httpx.Client(timeout=timeout, proxy=proxy_url) if proxy_url else httpx.Client(timeout=timeout)

research_instructions = """You are an expert researcher. Your job is to conduct thorough research, and then write a polished report."""

In [ ]:
"""Utility functions for displaying messages and prompts in Jupyter notebooks."""

import json

from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()


def format_message_content(message):
    """Convert message content to displayable string."""
    parts = []
    tool_calls_processed = False

    # Handle main content
    if isinstance(message.content, str):
        parts.append(message.content)
    elif isinstance(message.content, list):
        # Handle complex content like tool calls (Anthropic format)
        for item in message.content:
            if item.get("type") == "text":
                parts.append(item["text"])
            elif item.get("type") == "tool_use":
                parts.append(f"\n🔧 Tool Call: {item['name']}")
                parts.append(f"   Args: {json.dumps(item['input'], indent=2, ensure_ascii=False)}")
                parts.append(f"   ID: {item.get('id', 'N/A')}")
                tool_calls_processed = True
    else:
        parts.append(str(message.content))

    # Handle tool calls attached to the message (OpenAI format) - only if not already processed
    if (
        not tool_calls_processed
        and hasattr(message, "tool_calls")
        and message.tool_calls
    ):
        for tool_call in message.tool_calls:
            parts.append(f"\n🔧 Tool Call: {tool_call['name']}")
            parts.append(f"   Args: {json.dumps(tool_call['args'], indent=2, ensure_ascii=False)}")
            parts.append(f"   ID: {tool_call['id']}")

    return "\n".join(parts)


def format_messages(messages):
    """Format and display a list of messages with Rich formatting."""
    for m in messages:
        msg_type = m.__class__.__name__.replace("Message", "")
        content = format_message_content(m)

        if msg_type == "Human":
            console.print(Panel(content, title="🧑 Human", border_style="blue"))
        elif msg_type == "Ai":
            console.print(Panel(content, title="🤖 Assistant", border_style="green"))
        elif msg_type == "Tool":
            console.print(Panel(content, title="🔧 Tool Output", border_style="yellow"))
        else:
            console.print(Panel(content, title=f"📝 {msg_type}", border_style="white"))


def format_message(messages):
    """Alias for format_messages for backward compatibility."""
    return format_messages(messages)


def show_prompt(prompt_text: str, title: str = "Prompt", border_style: str = "blue"):
    """Display a prompt with rich formatting and XML tag highlighting.

    Args:
        prompt_text: The prompt string to display
        title: Title for the panel (default: "Prompt")
        border_style: Border color style (default: "blue")
    """
    # Create a formatted display of the prompt
    formatted_text = Text(prompt_text)
    formatted_text.highlight_regex(r"<[^>]+>", style="bold blue")  # Highlight XML tags
    formatted_text.highlight_regex(
        r"##[^#\n]+", style="bold magenta"
    )  # Highlight headers
    formatted_text.highlight_regex(
        r"###[^#\n]+", style="bold cyan"
    )  # Highlight sub-headers

    # Display in a panel for better presentation
    console.print(
        Panel(
            formatted_text,
            title=f"[bold green]{title}[/bold green]",
            border_style=border_style,
            padding=(1, 2),
        )
    )

# more expressive runner
async def stream_agent(agent, query, config=None):
    async for graph_name, stream_mode, event in agent.astream(
        query,
        stream_mode=["updates", "values"], 
        subgraphs=True,
        config=config
    ):
        if stream_mode == "updates":
            print(f'Graph: {graph_name if len(graph_name) > 0 else "root"}')
            
            node, result = list(event.items())[0]
            print(f'Node: {node}')
            
            for key in result.keys():
                if "messages" in key:
                    # print(f"Messages key: {key}")
                    format_messages(result[key])
                    break
        elif stream_mode == "values":
            current_state = event

    return current_state


In [24]:
import os
os.environ["OPENAI_USE_CHAT_COMPLETIONS_API"] = "true"
from langchain_openai import ChatOpenAI

model_gpt_5 = ChatOpenAI(
    model="gpt-5",
    base_url="https://agentrouter.org/v1",
    openai_api_key=os.getenv("AGENTROUTER_API_KEY"),
    default_headers={
        # Authorization проставится автоматически по openai_api_key
        "HTTP-Referer": "https://github.com/RooVetGit/Roo-Cline",
        "X-Title": "Roo Code",
        "User-Agent": "RooCode/1.0.0",
    },
)

agent = create_deep_agent(
    model=model_gpt_5,
    system_prompt=research_instructions,
)

# result = agent.invoke({"input": "Создай мне тренировку."})
result = agent.invoke({"messages": [{"role": "user", "content": "Создай мне тренировку."}]})

In [25]:
from utils.utils import format_messages

format_messages(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Создай мне тренировку.                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ С удовольствием! Чтобы составить программу именно под вас, ответьте, пожалуйста, на несколько вопросов:         │
│                                                                                                                 │
│ - Цель: жиросжигание, набор мышц, сила, выносливость, здоровье/осанка?                                          │
│ - Опыт: новичок/средний/продвинутый? Какие упражнения уже уверенно делаете (присед, жим, тяга, подтягивания)?   │
│ - Оборудование: дома/зал? Штанга, гантели, тренажёры, турник, резинки — что доступно?                           │
│ - Ограничения и травмы: спина, колени, плечи, давление и т. п.?                                                 │
│ - Расписание: сколько дней в неделю и по сколько минут на тренировку (например, 3×45–60 мин)?                   │
│ - Предпочтения: силовая/кардио/HIIT/йога/микс? Дом/улица/зал? Что нравится/не нравится?                         │
│ - Текущие данные и цели в цифрах (по желанию): возраст, рост/вес, шаги/дистанция, цель (например, −5 кг за 12   │
│ недель или 5 подтягиваний).                                                                                     │
│                                                                                                                 │
│ Хотите, пока вы отвечаете, набросаю базовый стартовый вариант на 3 дня (фулл-боди) без оборудования или с       │
│ гантелями — на ваш выбор?                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯